#  import

In [3]:
import random
import pandas as pd
import numpy as np
import glob
import os
import cv2

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, sampler
import torchvision.models as models

from tqdm.auto import tqdm

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from torchvision import datasets
from torchvision import transforms
import torchvision.models as models

from sklearn.metrics import f1_score

from IPython.display import Image
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore') 

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

# 하이퍼파라미터 세팅

In [5]:
CFG = {
    'IMG_SIZE':384,
    'EPOCHS':20,
    'LEARNING_RATE':0.001,
    'BATCH_SIZE':16,
    'SEED':42
}

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(CFG['SEED']) # Seed 고정

# 데이터 전처리

In [11]:
df = pd.read_csv('/kaggle/input/artist-data/train.csv')
df.loc[(df['id'] == 3896) & (df['artist'] == 'Titian'), ['img_path', 'id', 'artist']] = ['./train/3986.jpg', 3986, 'Alfred Sisley']
df.loc[(df['id'] == 3896) & (df['artist'] == 'Edgar Degas'), 'artist'] = 'Titian'
df.to_csv('/kaggle/working/new_train_data.csv', index=False)
df = pd.read_csv('/kaggle/working/new_train_data.csv')
df.head()

,id,img_path,artist
0,0,./train/0000.jpg,Diego Velazquez
1,1,./train/0001.jpg,Vincent van Gogh
2,2,./train/0002.jpg,Claude Monet
3,3,./train/0003.jpg,Edgar Degas
4,4,./train/0004.jpg,Hieronymus Bosch


In [12]:
encoder = preprocessing.LabelEncoder()
df['artist'] = encoder.fit_transform(df['artist'].values)
df.head(3)

,id,img_path,artist
0,0,./train/0000.jpg,9
1,1,./train/0001.jpg,48
2,2,./train/0002.jpg,7


In [13]:
train_df, val_df, _, _ = train_test_split(df, df['artist'], test_size=0.2,
                                          random_state=CFG['SEED'])

In [14]:
train_df = train_df.sort_values(by=['id'])
val_df = val_df.sort_values(by=['id'])

# display(train_df.head(3))
# display(val_df.head(3))

train_df.shape, val_df.shape

((4728, 3), (1183, 3))

# DataLoader

In [15]:
def get_data(df, infer=False):
    if infer:
        return df['img_path'].values
    return df['img_path'].values, df['artist'].values

In [18]:
train_img_paths, train_labels = get_data(train_df) # 4728개
val_img_paths, val_labels = get_data(val_df) # 1183개

len(train_img_paths), len(train_labels), len(val_img_paths), len(val_labels)

(4728, 4728, 1183, 1183)

# 데이터셋

In [19]:
class CustomDataset(Dataset):
    def __init__(self, img_paths, labels, transforms=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transforms = transforms
    
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, index):
        img_path = self.img_paths[index]
        img_path = img_path.replace('./t', '/t')
        img_path = ('/kaggle/input/artist-data' + img_path)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.labels is not None:
            label = self.labels[index]
            return image, label
        else:
            return image

In [20]:
train_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE']*2, CFG['IMG_SIZE']*2),
    A.RandomCrop(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.HorizontalFlip(p=0.5), 
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, # -0.1~0.1 범위에서 무작위로 이동
                               scale_limit=0.2, # -0.2~0.2 범위에서 무작위로 스케일
                               rotate_limit=30, # -30~30도 범위에서 무작위로 회전
                               p=0.5),
    A.OneOf([
             A.MotionBlur(p=1),
             A.OpticalDistortion(p=1),
             A.GaussNoise(p=1),
             A.CLAHE(clip_limit=2, tile_grid_size=(8,8), p=1),
             A.Rotate(limit=(45, 90), p=1, border_mode=cv2.BORDER_CONSTANT)
            ], p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225),
                max_pixel_value=255.0, always_apply=False, p=1),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE']*2,CFG['IMG_SIZE']*2),
    A.RandomCrop(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225),
                max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()
])

In [21]:
def make_weights(labels, nclasses):
    labels = np.array(labels)
    weight_arr = np.zeros_like(labels)

    _, counts = np.unique(labels, return_counts=True)
    for cls in range(nclasses):
        weight_arr = np.where(labels == cls, 1/counts[cls], weight_arr)
        # 각 클래스의의 인덱스를 산출하여 해당 클래스 개수의 역수를 확률로 할당한다.
        # 이를 통해 각 클래스의 전체 가중치를 동일하게 한다.

    return weight_arr

weights = make_weights(train_labels, len(np.unique(train_labels)))
weights = torch.DoubleTensor(weights)
print(weights, weights.shape)

tensor([0.0164, 0.0020, 0.0217,  ..., 0.0056, 0.0056, 0.0152],
       dtype=torch.float64) torch.Size([4728])


In [22]:
train_dataset = CustomDataset(train_img_paths, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], num_workers=0, 
                          sampler=sampler.WeightedRandomSampler(weights, len(weights)))

val_dataset = CustomDataset(val_img_paths, val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

# 모델 설정

## EfficienceNet_V2_M

In [23]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=50):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_v2_m(weights=True)
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features=1000, out_features=num_classes),
        )

    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [24]:
Eff_V2 = BaseModel(num_classes=50)
Eff_V2

Downloading: "https://download.pytorch.org/models/efficientnet_v2_m-dc08266a.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_m-dc08266a.pth


  0%|          | 0.00/208M [00:00<?, ?B/s]

BaseModel(
  (backbone): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): FusedMBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        )
        (1): FusedMBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): BatchNorm2d(2

# 모델 훈련

In [25]:
def competition_metric(true, pred):
    return f1_score(true, pred, average="macro")

In [26]:
def validation(model, criterion, test_loader, device):
    model.eval()
    
    model_preds = []
    true_labels = []
    
    val_loss = []
    
    with torch.no_grad():
        for img, label in tqdm(iter(test_loader)):
            img, label = img.float().to(device), label.to(device)
            
            model_pred = model(img)
            
            loss = criterion(model_pred, label)
            
            val_loss.append(loss.item())
            
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += label.detach().cpu().numpy().tolist()
        
    val_f1 = competition_metric(true_labels, model_preds)
    return np.mean(val_loss), val_f1

In [27]:
def train(model, optimizer, train_loader, test_loader, scheduler, device):
    model.to(device)
    
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS'] + 1):
        model.train()
        train_loss = []
        
        for img, label in tqdm(iter(train_loader)):
            img = img.float().to(device)
            label = label.to(device)
            
            optimizer.zero_grad()
            
            model_pred = model(img)
            
            loss = criterion(model_pred, label)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        tr_loss = np.mean(train_loss)
        
        val_loss, val_score = validation(model, criterion, test_loader, device)
        
        print(f'Epoch [{epoch}], Train Loss : [{tr_loss:.5f}] Val Loss : [{val_loss:.5f}] Val F1 Score : [{val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(metrics=val_score)
        
        if best_score < val_score:
            best_model = model
            best_score = val_score
    return best_model

In [28]:
Eff_V2.eval()
optimizer_Eff_V2 = torch.optim.Adam(params=Eff_V2.parameters(), lr=CFG['LEARNING_RATE'])
scheduler_Eff_V2 = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer_Eff_V2,
                                                 mode='max', factor=0.1,
                                                 patience=1, verbose=True)

In [29]:
model = train(Eff_V2, optimizer_Eff_V2, train_loader, val_loader,
                    scheduler_Eff_V2, device=device)

  0%|          | 0/296 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

Epoch [1], Train Loss : [3.42533] Val Loss : [3.31936] Val F1 Score : [0.08885]


  0%|          | 0/296 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

...

  0%|          | 0/74 [00:00<?, ?it/s]

Epoch [19], Train Loss : [0.70270] Val Loss : [1.05668] Val F1 Score : [0.67181]


  0%|          | 0/296 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

Epoch [20], Train Loss : [0.66246] Val Loss : [1.06703] Val F1 Score : [0.68354]


# 모델 평가

In [30]:
test_df = pd.read_csv('/kaggle/input/artist-data/test.csv')
test_df.head()

,id,img_path
0,TEST_00000,./test/TEST_00000.jpg
1,TEST_00001,./test/TEST_00001.jpg
2,TEST_00002,./test/TEST_00002.jpg
3,TEST_00003,./test/TEST_00003.jpg
4,TEST_00004,./test/TEST_00004.jpg


In [31]:
test_img_paths = get_data(test_df, infer=True)

In [32]:
test_dataset = CustomDataset(test_img_paths, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [33]:
def evaluation(model, test_loader, device):
    model.to(device)
    model.eval()
    
    model_preds = []
    
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            img = img.float().to(device)
            
            model_pred = model(img)
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
    
    print('Done.')
    return model_preds

In [34]:
preds = evaluation(model, test_loader, device)

  0%|          | 0/792 [00:00<?, ?it/s]

Done.


In [35]:
preds = encoder.inverse_transform(preds) # LabelEncoder로 변환 된 Label을 다시 화가 이름으로 변환

In [37]:
submit = pd.read_csv('/kaggle/input/artist-data/sample_submission.csv')
submit.head()

,id,artist
0,TEST_00000,Edgar Degas
1,TEST_00001,Edgar Degas
2,TEST_00002,Edgar Degas
3,TEST_00003,Edgar Degas
4,TEST_00004,Edgar Degas


In [38]:
submit['artist'] = preds
submit.head()

,id,artist
0,TEST_00000,Edgar Degas
1,TEST_00001,Amedeo Modigliani
2,TEST_00002,Caravaggio
3,TEST_00003,Albrecht Du rer
4,TEST_00004,Pablo Picasso


In [39]:
submit.to_csv('/kaggle/working/efficientnet_v2_m_2.csv', index=False)

# 모델 저장

In [40]:
model.state_dict().keys()

odict_keys(['backbone.features.0.0.weight', 'backbone.features.0.1.weight', 'backbone.features.0.1.bias', 'backbone.features.0.1.running_mean', 'backbone.features.0.1.running_var', 'backbone.features.0.1.num_batches_tracked', 'backbone.features.1.0.block.0.0.weight', 'backbone.features.1.0.block.0.1.weight', 'backbone.features.1.0.block.0.1.bias', 'backbone.features.1.0.block.0.1.running_mean', 'backbone.features.1.0.block.0.1.running_var', 'backbone.features.1.0.block.0.1.num_batches_tracked', 'backbone.features.1.1.block.0.0.weight', 'backbone.features.1.1.block.0.1.weight', 'backbone.features.1.1.block.0.1.bias', 'backbone.features.1.1.block.0.1.running_mean', 'backbone.features.1.1.block.0.1.running_var', 'backbone.features.1.1.block.0.1.num_batches_tracked', 'backbone.features.1.2.block.0.0.weight', 'backbone.features.1.2.block.0.1.weight', 'backbone.features.1.2.block.0.1.bias', 'backbone.features.1.2.block.0.1.running_mean', 'backbone.features.1.2.block.0.1.running_var', 'backbo

In [41]:
torch.save(model.state_dict(), '/kaggle/working/efficientnet_v2_m_2.pth')